In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time
import os
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings(action='ignore')
from scipy.signal import butter, lfilter, freqz
# from torch import nn, optim
import sqlite3, os
import math
import random
import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.losses import Huber, binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
pd.set_option('display.max_rows',100)

In [ ]:
path = 'SisFall_dataset/'
person = ['SA01','SA02','SA03','SA04','SA05','SA06','SA07','SA08','SA09','SA10','SA11','SA12','SA13','SA14','SA15','SA16','SA17','SA18','SA19','SA20']

person2 = ['SA21','SA22','SA23']
# person = ['data1','data3','data5','data7','data9','data11','data13','data15','data17','data19','data21','data23']
# person = os.listdir(path)
# person.pop(0)

In [ ]:
# dailies = ['D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15',
#            'D16', 'D17', 'D18', 'D19']
dailies = ['D04', 'D06']
falls = ['F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15']
# falls = ['F01', 'F02', 'F03', 'F04']
trials = ['R01', 'R02','R03','R04','R05']

path = '/content/drive/MyDrive/sisfall_data/'

In [ ]:

# Filter requirements.
order = 4
fs = 200.0  # sample rate, Hz
cutoff = 5.0  # desired cutoff frequency of the filter, Hz


# From??????
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a


def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
def printer(df, min_max=True, range=[0,3000], *args):
    idx = {}
    if min_max:
        x = df.drop(['activity','person'],axis=1).astype('float64')
        k = (x - x.min()) / (x.max() - x.min())
        for w in args:
            plt.plot(k[w],label=w)
            idx[w] = df[w][df[w] == df[w].max()].index.values
        plt.title(f'person:{df.person[0]}  ,  activity:{df.activity[0]}')
        plt.xlim(range)
        plt.legend()
        plt.show()
        for w in idx:
            print(f'{w} : {idx[w]}')
    else:
        for w in args:
            plt.plot(df[w],label=w)
        plt.title(df.activity[0])
        plt.xlim(range)
        plt.legend()
        plt.show()

In [ ]:
def preprocess(person):
    subjectList = []
    adl_list = []
    fall_list = []
    for man in person:
        a = pd.read_csv(path + f'{man}.csv')
        df = a[['ADXL_x', 'ADXL_y', 'ADXL_z', 'ITG_x', 'ITG_y', 'ITG_z', 'subject', 'activity', 'trial']]
    #     df['ADXL_x'] *= (32/8192)
    #     df['ADXL_y'] *= (32/8192)
    #     df['ADXL_z'] *= (32/8192)
        subjectList.append(df)
    for s in subjectList:
        for d in dailies:
            tempdf = s[s['activity'] == d]
            adl_list.append(tempdf)

        for f in falls:
            tempdf = s[s['activity'] == f]
            fall_list.append(tempdf)
            
    return fall_list, adl_list
    

In [ ]:
def preprocess2(df):
    df_list = []
    rol_val = 50
    for my_df in df:
        for trial in trials:
            trial_df = my_df[my_df['trial'] == trial]
            if len(trial_df) == 0:
                continue
            trial_df.reset_index(inplace=True)
            tempdf = pd.DataFrame()
            tempdf['ax'], tempdf['ay'], tempdf['az'] = trial_df['ADXL_x'], trial_df['ADXL_y'], trial_df['ADXL_z']
            tempdf = tempdf.reset_index(drop=True)
            tempdf['fx'] = pd.Series(butter_lowpass_filter(trial_df['ADXL_x'], cutoff, fs, order))
            tempdf['fy'] = pd.Series(butter_lowpass_filter(trial_df['ADXL_y'], cutoff, fs, order))
            tempdf['fz'] = pd.Series(butter_lowpass_filter(trial_df['ADXL_z'], cutoff, fs, order))
            tempdf['vs'] = np.sqrt(tempdf['ax'] ** 2 + tempdf['ay'] ** 2 + tempdf['az'] ** 2)
            tempdf['vm2'] =np.sqrt(tempdf['fx'] ** 2 + tempdf['fy'] ** 2 + tempdf['fz'] ** 2)

            tempdf['bx'] = tempdf['fx'].diff()
            tempdf['by'] = tempdf['fy'].diff()
            tempdf['bz'] = tempdf['fz'].diff()
            tempdf['diff_sum'] = np.sqrt(tempdf['bx'] ** 2 + tempdf['by'] ** 2 + tempdf['bz'] ** 2)
            tempdf['activity'] = trial_df['activity']
            tempdf['person'] = trial_df['subject']

            tempdf = tempdf.reset_index(drop=True)
            trial_df = trial_df.reset_index(drop=True)
            tempdf['gx'], tempdf['gy'], tempdf['gz'] = trial_df['ITG_x'], trial_df['ITG_y'], trial_df['ITG_z']
            tempdf['g_fx'] = pd.Series(butter_lowpass_filter(tempdf['gx'], cutoff, fs, order))
            tempdf['g_fy'] = pd.Series(butter_lowpass_filter(tempdf['gy'], cutoff, fs, order))
            tempdf['g_fz'] = pd.Series(butter_lowpass_filter(tempdf['gz'], cutoff, fs, order))
            tempdf['g_sum'] = np.sqrt(tempdf['g_fx'] ** 2 + tempdf['g_fy'] ** 2 + tempdf['g_fz'] ** 2)


            #Rolling standard deviations
            tempdf['bx_std'] = tempdf['bx'].rolling(rol_val,min_periods = 1).std()
            tempdf['by_std'] = tempdf['by'].rolling(rol_val,min_periods = 1).std()
            tempdf['bz_std'] = tempdf['bz'].rolling(rol_val,min_periods = 1).std()
            tempdf['fx_std'] = tempdf['fx'].rolling(rol_val,min_periods = 1).std()
            tempdf['fy_std'] = tempdf['fy'].rolling(rol_val,min_periods = 1).std()
            tempdf['fz_std'] = tempdf['fz'].rolling(rol_val,min_periods = 1).std()
            tempdf['gx_std'] = tempdf['gx'].rolling(rol_val,min_periods = 1).std()
            tempdf['gy_std'] = tempdf['gy'].rolling(rol_val,min_periods = 1).std()
            tempdf['gz_std'] = tempdf['gz'].rolling(rol_val,min_periods = 1).std()

            # had to make versions of this to put into sliding window, will change once I
            # confirm they're the same as the others below
            #C8
            tempdf['horiz_std_mag9'] = np.sqrt(tempdf['fx_std'] ** 2 + tempdf['fz_std'] ** 2)
            #C2
            tempdf['horiz_vector_mag9'] = np.sqrt(tempdf['fx'] ** 2 + tempdf['fz'] ** 2)
            tempdf['std_mag9'] = np.sqrt(tempdf['fx_std'] ** 2 + tempdf['fy_std'] ** 2 + tempdf['fz_std'] ** 2)
            #C9
            tempdf['horiz_std_mag2'] = np.sqrt(tempdf['bx_std'] ** 2 + tempdf['bz_std'] ** 2)
            tempdf['gyro_horiz_std_mag'] = np.sqrt(tempdf['gx_std'] ** 2 + tempdf['gz_std'] ** 2)
            tempdf['gyro_vector_mag'] = np.sqrt(tempdf['gx'] ** 2 + tempdf['gy'] ** 2 + tempdf['gz'] ** 2)
            tempdf['gyro_horiz_mag'] = np.sqrt(tempdf['gx'] ** 2 + tempdf['gz'] ** 2)
            # C2
            tempdf['horiz_mag'] = np.sqrt(tempdf['fx'] ** 2 + tempdf['fz'] ** 2)

            df_list.append(tempdf.fillna(0))
    return df_list

In [ ]:
def labeling(fall_list,adl_list, standard,min_max, check_list):
    adl_df = []
    fall_df = []

    for df in fall_list:
        df = df.loc[:,check_list]
        if min_max:
            df = (df - df.min()) / (df.max() - df.min())

        k = df[standard].idxmax()
        fall = df.loc[k - 120 : k]
        fall['label'] = 1
        fall_df.append(fall)
    
    for df in adl_list:
        df = df.loc[120:,check_list]
        if min_max:
            df = (df - df.min()) / (df.max() - df.min())
        df['label'] = 0
        adl_df.append(df)

    return fall_df, adl_df

In [ ]:
def windowing(fall_df,adl_df , WINDOWSIZE):
    x = []
    y = []
    for df in fall_df:
        for i in range(len(df) - WINDOWSIZE):
            df_x = df.drop('label',axis=1).iloc[i : i+ WINDOWSIZE]
            df_y = df['label'].iloc[1]
            x.append(df_x)
            y.append(df_y)
    for df in adl_df:
        count = 0
        for i in range(120,len(df) - WINDOWSIZE,20):
            count += 1
            df_x = df.drop('label',axis=1).iloc[i : i+ WINDOWSIZE]
            df_y = df['label'].iloc[1]
            x.append(df_x)
            y.append(df_y)
            if count > 120:
                break
    x = np.array(x)
    y = np.array(y)
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state = 0)
    return x_train, x_valid, y_train, y_valid

In [ ]:
WINDOWSIZE = 60

In [ ]:
# preprocess
fall_list, adl_list = preprocess(person)
fall_list = preprocess2(fall_list)
adl_list = preprocess2(adl_list)

In [ ]:
checkList = checkList = [
'std_mag9',
'horiz_std_mag2',
'gyro_vector_mag',
'gyro_horiz_mag',
'vm2'
]

In [ ]:
count = 0

fig, ax = plt.subplots(6,5,figsize=(30,30))
for k in range(0, 60, 10):
    count2 = 0
    df = fall_list[k][checkList]
    df = (df - df.min()) / (df.max() - df.min())
    for i in checkList:

        ax[count,count2].plot(df[i], label=i)
        ax[count,count2].plot(df['vm2'] ,label = 'vm2')
        ax[count,count2].legend()
        count2 += 1
    count += 1

plt.show()


In [ ]:
fall_df, adl_df = labeling(fall_list,adl_list,'vm2',1, ['g_sum','diff_sum','vm2'])

In [ ]:
x_train, x_valid, y_train, y_valid = windowing(fall_df , adl_df, 60)

In [ ]:
# model = tf.keras.models.Sequential([
#     # 1차원 feature map 생성
# #     tf.keras.layers.Conv1D(filters=32, kernel_size=5,
# #            padding="causal",
# #            activation="relu",dff
# #            input_shape=[WINDOW_SIZE, 4]),
#     # LSTM
#     tf.keras.layers.LSTM(128,input_shape=[WINDOWSIZE, 3], activation='tanh', return_sequences=True ),
#     tf.keras.layers.LSTM(64,  return_sequences=True ),
#     tf.keras.layers.LSTM(32),
#     tf.keras.layers.Dense(16, activation="relu"),
#     tf.keras.layers.Dense(8, activation="relu"),
#     tf.keras.layers.Dense(1, activation="sigmoid"),
# ])

In [ ]:
model = tf.keras.models.Sequential([
    # 1차원 feature map 생성
#     tf.keras.layers.Conv1D(filters=32, kernel_size=5,
#            padding="causal",
#            activation="relu",dff
#            input_shape=[WINDOW_SIZE, 4]),
    # LSTM
    tf.keras.layers.LSTM(128,input_shape=[WINDOWSIZE, 3], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(64,  return_sequences=True,dropout=0.2 ),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics='accuracy')
early_stop = EarlyStopping(monitor='val_loss', patience=15)
filename = os.path.join(path + 'tmp_checkpoint.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit(x_train, y_train, 
                    epochs=100, 
                    batch_size=32,
                    validation_data=(x_valid, y_valid), 
                    callbacks=[early_stop, checkpoint])

In [ ]:
model = keras.models.load_model('/content/tmp_checkpoint.h5')

In [ ]:
score = model.evaluate(x_train,y_train)
print(score[1])
print(score[0])
# p = pd.DataFrame(pre.round())
# p
# x = pd.DataFrame(y_train)
# t = pd.concat([p,x],axis=1)
# (t.iloc[:,1] + t.iloc[:,0]).value_counts()

In [ ]:
fall_list, adl_list = preprocess(person2)
fall_list = preprocess2(fall_list)
adl_list = preprocess2(adl_list)

In [ ]:
adl_list[10][['vm2','activity','person']]

In [ ]:
##test data
fall_df = []
adl_df = []
for df in fall_list:
    df = df[['g_sum','diff_sum','vm2']]
    df = (df - df.min()) / (df.max() - df.min())
    df['label'] = 0
    k = df.g_sum.idxmax()
    df['label'].loc[k-60:k] = 1
    df = df.loc[k-120:k]
    fall_df.append(df)
for df in adl_list:
    df = df[['g_sum','diff_sum','vm2']]
    df = (df - df.min()) / (df.max() - df.min())
    df['label'] = 0
    df = df.loc[200:400]
    adl_df.append(df)

test_x = []
test_y = []
WINDOWSIZE = 60
for df in fall_df:
    for i in range(len(df) - WINDOWSIZE):
        df_x = df.drop('label',axis=1).iloc[i : i+ WINDOWSIZE]
        df_y = df['label'].iloc[i+WINDOWSIZE]
        test_x.append(df_x)
        test_y.append(df_y)

for df in adl_df:
    for i in range(0, len(df) - WINDOWSIZE):
        df_x = df.drop('label',axis=1).iloc[i : i+ WINDOWSIZE]
        df_y = df['label'].iloc[i+WINDOWSIZE]
        test_x.append(df_x)
        test_y.append(df_y)

np_x = np.array(test_x)
np_y = np.array(test_y)
    

In [ ]:
score = model.evaluate(np_x,np_y)


509/509 [==============================] - 8s 6ms/step - loss: 0.1982 - accuracy: 0.9437


In [ ]:
score

[0.19824744760990143, 0.9436758160591125]